<a href="https://colab.research.google.com/github/souravs17031999/private-ai/blob/master/final_secured_fed_securing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Objective : Perform Federated Learning with Encrypted Gradient Aggregation

In [0]:
!pip install syft

  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067079 sha256=23fa89ab0fc87772716cac4936aff07524a8dfae012928d16d1c864d6ccff34d
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=3c8f21922c450af91b915afe606b635f8c3f7655b20b5c769af4666a7bf9e12a
  Stored in directory: /root/.cache/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
      Successfully uninstalled msgpack-0.5.6


In [0]:
import torch as th
import syft as sy
from torch import nn, optim
hook = sy.TorchHook(th)

In [0]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

W0805 14:20:32.794059 139685491410816 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0805 14:20:32.800300 139685491410816 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0805 14:20:32.805066 139685491410816 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0805 14:20:32.807551 139685491410816 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0805 14:20:32.810148 139685491410816 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0805 14:20:32.812270 139685491410816 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


In [0]:
model = nn.Linear(2,1)
print(model)


Linear(in_features=2, out_features=1, bias=True)


In [34]:
epochs = 10
iterations = 5

for e in range(1, epochs + 1):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_optimizer = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_optimizer = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for _ in range(iterations):
        # Train Bob's Model
        bobs_optimizer.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_optimizer.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_optimizer.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_optimizer.step()
        alices_loss = alices_loss.get().data

    alices_model.get()
    bobs_model.get()
    with th.no_grad():
      model.weight.set_(((alices_model.weight.data.fix_prec().share(bob, alice, secure_worker) + bobs_model.weight.data.fix_prec().share(bob, alice, secure_worker)) / 2).get().float_prec())
      model.bias.set_(((alices_model.bias.data.fix_prec().share(bob, alice, secure_worker) + bobs_model.bias.data.fix_prec().share(bob, alice, secure_worker)) / 2).get().float_prec())
        
    print(f"{e}/epochs completed")
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

1/epochs completed
Bob:tensor(2.4372e-07) Alice:tensor(2.2928e-07)
2/epochs completed
Bob:tensor(2.4372e-07) Alice:tensor(2.2928e-07)
3/epochs completed
Bob:tensor(2.4372e-07) Alice:tensor(1.2897e-07)
4/epochs completed
Bob:tensor(2.4372e-07) Alice:tensor(5.7331e-08)
5/epochs completed
Bob:tensor(7.4140e-07) Alice:tensor(3.4936e-08)
6/epochs completed
Bob:tensor(2.4372e-07) Alice:tensor(1.4341e-08)
7/epochs completed
Bob:tensor(3.4511e-08) Alice:tensor(4.4321e-07)
8/epochs completed
Bob:tensor(6.0931e-08) Alice:tensor(5.7308e-08)
9/epochs completed
Bob:tensor(6.0931e-08) Alice:tensor(1.4330e-08)
10/epochs completed
Bob:tensor(0.) Alice:tensor(5.7331e-08)


In [35]:
preds = model(data)
loss = ((preds - target) ** 2).sum()
print(preds)
print(target)
print(loss.data)

tensor([[0.0000],
        [0.0010],
        [0.9980],
        [0.9990]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(5.9999e-06)
